In [107]:
import pandas as pd
import unidecode
import re

import nltk
from nltk.corpus import stopwords

gmail = pd.read_csv("data/gmail_oscar.CSV")

zimbra = pd.read_csv("data/zimbra_oscar.CSV")

i = 0
for corps in gmail["Corps"]:
    text_file = open(r'corps.txt', 'w',  encoding="utf-8")
    text_file.write(corps)
    text_file.close()

    
    final_text_file = open(r'final_corps.txt', 'w',  encoding="utf-8")
    reading_text_file = open(r'corps.txt', 'r',  encoding="utf-8")
    for line in reading_text_file:
        if "https" not in line:
            final_text_file.write(line)
        
    final_text_file.close()
    reading_text_file.close()

    with open('final_corps.txt', 'r', encoding="utf-8") as file:
        data = file.read()
    

    data = unidecode.unidecode(data)
    data = data.lower()
    data = re.sub(r"""[.,"'-:!;]""", '', data)
    data = re.sub('\s+', ' ', data)
    data = data.split(' ')


    # print("avant de retirer les stopwords : ", len(data), " mots")
    stop_words = stopwords.words('french')
    data = [word for word in data if word not in stopwords.words('french') and len(word)>1]
    # print("Après avoir retiré les stopwords : ", len(data), " mots")

    gmail["Corps"][i] = data

    i = i+1

print(zimbra.columns)

#things to do : lemmatizer + faire ca pour zimbra aussi (ou alors combiner nos datasets pour en faire qu'un seul dès le début) + faire le meme process de traitement cleaning sur l'objet du mail aussi par exemple.

<ipython-input-107-b0269a0bd22e>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gmail["Corps"][i] = data


Index(['Objet', 'Corps', 'De: (nom)', 'De: (adresse)', 'De: (type)',
       'A: (nom)', 'A: (adresse)', 'A: (type)', 'Cc: (nom)', 'Cc: (adresse)',
       'Cc: (Type)', 'Cci: (nom)', 'Cci: (adresse)', 'Cci: (type)',
       'Catégories', 'Critère de diffusion', 'Importance',
       'Informations facturation', 'Kilométrage'],
      dtype='object')
